# template_engine

> Fill in a module description here

In [ ]:
#| default_exp template_engine

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from fastcore.basics import patch_to
from fastcore.test import *
from typing import List, Tuple, Dict, Any, Callable, Optional
from tk_slack.core import ValueFormatter, DebugLogger, ColumnUtils, SlackFormatter
from tk_slack.block_builder import BlockBuilder
from tk_slack.interaction_builder import InteractionBuilder
import pandas as pd
import json

In [ ]:
#| export

class TemplateEngine:
    """
    Core template engine for creating Slack messages from DataFrame data.
    Provides methods to generate messages in different formats.
    """
    pass

In [ ]:
#| export

@patch_to(TemplateEngine,cls_method=True)
def _parse_row_config(self, row: pd.Series, view_config: Dict[str, Any], 
                         col_map: Dict[str, str]) -> Dict[str, Any]:
        """Parse row-specific configuration, falling back to view config.
        
        Args:
            row: DataFrame row
            view_config: View-level configuration
            col_map: Column name mapping (uppercase to original case)
            
        Returns:
            Merged configuration
        """
        # Start with the view config
        config = view_config.copy()
        
        # Check for row-specific config
        if 'CONFIG' in col_map and pd.notna(row[col_map['CONFIG']]):
            try:
                row_config = row[col_map['CONFIG']]
                if not isinstance(row_config, dict):
                    row_config = json.loads(row_config)
                # Merge with view_config, with row_config taking precedence
                config.update(row_config)
            except (json.JSONDecodeError, TypeError):
                DebugLogger.log(f"Error parsing row config. Using view_config.")
        
        return config

In [ ]:
#| export

@patch_to(TemplateEngine,cls_method=True)
def _extract_meta_data_fields(self, row: pd.Series, df_columns: List[str], 
                                 config: Dict[str, Any]) -> List[Tuple[str, str]]:
        """Extract metadata fields from a row.
        
        Args:
            row: DataFrame row
            df_columns: DataFrame column names
            config: Configuration dictionary
            
        Returns:
            List of (label, value) tuples for metadata
        """
        # Get metadata columns from config and columns ending with _meta
        meta_data_cols = config.get('meta_data_cols', [])
        meta_data_cols += [col for col in df_columns if col.lower().endswith('_meta')]
        
        meta_items = []
        for col in meta_data_cols:
            if col in df_columns and pd.notna(row[col]):
                # Format the field name and value
                field_name = col.lower().replace('_meta', '').replace('_', ' ').title()
                field_value = ValueFormatter.format_value(row[col])
                if field_value:
                    meta_items.append((field_name, field_value))
                    
        return meta_items

In [ ]:
#| export

@patch_to(TemplateEngine,cls_method=True)
def _extract_detail_fields(self, row: pd.Series, df_columns: List[str], 
                              config: Dict[str, Any]) -> List[Tuple[str, str]]:
        """Extract detail fields from a row.
        
        Args:
            row: DataFrame row
            df_columns: DataFrame column names
            config: Configuration dictionary
            
        Returns:
            List of (label, value) tuples for detail fields
        """
        # Get detail columns from config or use default
        detail_cols = config.get('detail_cols', [])
        if not detail_cols:
            # If not specified, use utility to get default detail columns
            detail_cols = ColumnUtils.get_detail_columns(df_columns)
        
        field_items = []
        for col in detail_cols:
            if col in df_columns and pd.notna(row[col]):
                # Format field name and value for display
                field_name = col.lower().replace('_', ' ').title()
                field_value = ValueFormatter.format_value(row[col])
                if field_value:
                    field_items.append((field_name, field_value))
                    
        return field_items

In [ ]:
#| export

@patch_to(TemplateEngine,cls_method=True)
def _extract_interactive_options(self, row: pd.Series, col_map: Dict[str, str]) -> Tuple[List[str], List[str]]:
        """Extract interactive option names and values from a row.
        
        Args:
            row: DataFrame row
            col_map: Column name mapping (uppercase to original case)
            
        Returns:
            Tuple of (option_names, option_values)
        """
        option_names = []
        option_values = []
        
        # Check for option_name in case-insensitive manner
        option_name_col = col_map.get('OPTION_NAME')
        option_value_col = col_map.get('OPTION_VALUE')
        
        if option_name_col and pd.notna(row[option_name_col]):
            option_names = row[option_name_col]
            option_values = row[option_value_col] if option_value_col and pd.notna(row[option_value_col]) else ''
            
            # Convert to lists if they're not already
            if not isinstance(option_names, list):
                option_names = [str(option_names)]
            
            if not isinstance(option_values, list):
                option_values = [str(option_values)] * len(option_names)
            elif len(option_values) < len(option_names):
                # Extend option_values if it's shorter than option_names
                option_values = option_values + [str(option_values[-1])] * (len(option_names) - len(option_values))
                
        return option_names, option_values

In [ ]:
#| export

@patch_to(TemplateEngine,cls_method=True)
def build_individual_message_blocks(cls, row: pd.Series, df_columns: List[str], 
                                       col_map: Dict[str, str], config: Dict[str, Any],
                                       view: str) -> List[Dict[str, Any]]:
        """Build message blocks for a single row.
        
        Args:
            row: DataFrame row
            df_columns: DataFrame column names
            col_map: Column name mapping
            config: Configuration dictionary
            view: View name for action IDs
            
        Returns:
            List of Slack blocks for the message
        """
        # Initialize blocks for this message
        payload_blocks = []
        
        # 1. Title Section - Use SlackFormatter for title with proper linking
        section_text = SlackFormatter.format_section_name(row, df_columns)
        payload_blocks.append(BlockBuilder.create_section_block(section_text))
        
        # 2. Description Text - Look for TEXT or DESCRIPTION column
        for text_col in ['TEXT', 'DESCRIPTION']:
            if text_col in col_map and pd.notna(row[col_map[text_col]]):
                payload_blocks.append(
                    BlockBuilder.create_section_block(str(row[col_map[text_col]]))
                )
                break
        
        # 3. Metadata - Get and format metadata fields
        meta_items = cls._extract_meta_data_fields(row, df_columns, config)
        if meta_items:
            meta_block = BlockBuilder.create_metadata_context(meta_items)
            if meta_block:
                payload_blocks.append(meta_block)
        
        # 4. Detail Fields - Get and format detail fields
        field_items = cls._extract_detail_fields(row, df_columns, config)
        if field_items:
            field_blocks = BlockBuilder.create_fields_section(field_items)
            payload_blocks.extend(field_blocks)
        
        # 5. Interactive Elements
        option_names, option_values = cls._extract_interactive_options(row, col_map)
        
        if option_names:
            action_type = config.get('action_type', None)
            action_id_base = f"{view}_{action_type if action_type else 'action'}"
            
            elements = InteractionBuilder.detect_and_create_interactive_elements(
                option_names, option_values, action_id_base, action_type
            )
            
            if elements:
                payload_blocks.append(
                    InteractionBuilder.create_actions_block(elements)
                )
        
        # 6. Add a divider at the end
        payload_blocks.append(BlockBuilder.create_divider())
        
        return payload_blocks

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()